In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

def change_name(element):
    if 'v1.0' in element:
        return 'v1.0'
    elif 'v1.1' in element:
        return 'v1.1'
    elif 'FT' in element:
        return 'FT'
    elif 'B4' in element:
        return 'B4'
    elif 'B5' in element:
        return 'B5'

spacex_df['Booster Version Category'] = spacex_df['Booster Version'].apply(change_name)

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div(dcc.Dropdown(
                                        id='site-dropdown',
                                        options=[{'label': 'All Sites', 'value': 'All'}, 
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}, 
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                        value= 'All'
                                        )),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                        min=0, max=10000, step=1000,
                                        marks={0: '0', 2500: '2500', 5000: '5000', 
                                            7500: '7500', 10000:'10000'},
                                        value=[min_payload, max_payload])),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Div(id='output-container')
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def update_pie_chart(input_site_name):
    if input_site_name == 'All':
        success_launch_sites_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        chart = px.pie(success_launch_sites_df,
                values='class',
                names='Launch Site',
                title="Total Success Launches By Site")
    
    elif input_site_name == 'CCAFS LC-40':
        df1 = spacex_df[spacex_df["Launch Site"] == 'CCAFS LC-40'][['Launch Site', 'class']]
        new_df1 = df1['class'].value_counts().to_frame().reset_index()
        new_df1.columns = ['class', 'S/F counts']
        chart = px.pie(new_df1,
                values='S/F counts',
                names='class',
                title="Total Success Launches for site CCAFS LC-40")

    elif input_site_name == 'CCAFS SLC-40':
        df1 = spacex_df[spacex_df["Launch Site"] == 'CCAFS SLC-40'][['Launch Site', 'class']]
        new_df1 = df1['class'].value_counts().to_frame().reset_index()
        new_df1.columns = ['class', 'S/F counts']
        chart = px.pie(new_df1,
                values='S/F counts',
                names='class',
                title="Total Success Launches for site CCAFS SLC-40")

    elif input_site_name == 'KSC LC-39A':
        df1 = spacex_df[spacex_df["Launch Site"] == 'KSC LC-39A'][['Launch Site', 'class']]
        new_df1 = df1['class'].value_counts().to_frame().reset_index()
        new_df1.columns = ['class', 'S/F counts']
        chart = px.pie(new_df1,
                values='S/F counts',
                names='class',
                title="Total Success Launches for site KSC LC-39A")

    elif input_site_name == 'VAFB SLC-4E':
        df1 = spacex_df[spacex_df["Launch Site"] == 'VAFB SLC-4E'][['Launch Site', 'class']]
        new_df1 = df1['class'].value_counts().to_frame().reset_index()
        new_df1.columns = ['class', 'S/F counts']
        chart = px.pie(new_df1,
                values='S/F counts',
                names='class',
                title="Total Success Launches for site VAFB SLC-4E")
    return chart

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    [Output(component_id='output-container', component_property='children'),
     Output(component_id='success-payload-scatter-chart', component_property='figure')],
    #Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
    Input(component_id="payload-slider", component_property="value")])

def update_slider(input_site_name, input_slider_value):
    if input_site_name == 'All':
        df = spacex_df[(spacex_df['Payload Mass (kg)'] >= input_slider_value[0]) & 
                (spacex_df['Payload Mass (kg)'] <= input_slider_value[1])]
        chart = px.scatter(df,
                x='Payload Mass (kg)',
                y='class',
                color= df['Booster Version Category'],
                title="Correlation Between Payload and Success for All Sites")
    
    elif input_site_name == 'CCAFS LC-40':
        df1 = spacex_df[(spacex_df["Launch Site"] == 'CCAFS LC-40') & 
                        (spacex_df['Payload Mass (kg)'] >= input_slider_value[0]) & 
                        (spacex_df['Payload Mass (kg)'] <= input_slider_value[1])]
        chart = px.scatter(df1,
                x='Payload Mass (kg)',
                y='class',
                color= df1['Booster Version Category'],
                title="Correlation Between Payload and Success for CCAFS LC-40")

    elif input_site_name == 'CCAFS SLC-40':
        df1 = spacex_df[(spacex_df["Launch Site"] == 'CCAFS SLC-40') & 
                        (spacex_df['Payload Mass (kg)'] >= input_slider_value[0]) & 
                        (spacex_df['Payload Mass (kg)'] <= input_slider_value[1])]
        chart = px.scatter(df1,
                x='Payload Mass (kg)',
                y='class',
                color= df1['Booster Version Category'],
                title="Correlation Between Payload and Success for CCAFS SLC-40")

    elif input_site_name == 'KSC LC-39A':
        df1 = spacex_df[(spacex_df["Launch Site"] == 'KSC LC-39A') & 
                        (spacex_df['Payload Mass (kg)'] >= input_slider_value[0]) & 
                        (spacex_df['Payload Mass (kg)'] <= input_slider_value[1])]
        chart = px.scatter(df1,
                x='Payload Mass (kg)',
                y='class',
                color= df1['Booster Version Category'],
                title="Correlation Between Payload and Success for KSC LC-39A")

    elif input_site_name == 'VAFB SLC-4E':
        df1 = spacex_df[(spacex_df["Launch Site"] == 'VAFB SLC-4E') & 
                        (spacex_df['Payload Mass (kg)'] >= input_slider_value[0]) & 
                        (spacex_df['Payload Mass (kg)'] <= input_slider_value[1])]
        chart = px.scatter(df1,
                x='Payload Mass (kg)',
                y='class',
                color= df1['Booster Version Category'],
                title="Correlation Between Payload and Success for VAFB SLC-4E")
        
    return f"You have selected a range from {input_slider_value[0]} to {input_slider_value[1]}", chart

# Run the app
if __name__ == '__main__':
    app.run_server()